In [149]:
import pandas as pd
import numpy as np
import plotly.express as px
import math as math

# Datos de entrada y calculados-WPI

In [150]:
# Entradas que deben ir en la app
p200 = 80
PI = 25
wPI = 20
# calculados wpi
a_wpi = 0.00364 * (wPI ** 3.35) + 4 * wPI + 11
c_wpi = 0.0514 * (wPI ** 0.465) + 0.5
b_wpi= c_wpi * (-2.313 * (wPI ** 0.14) + 5)
hr_wpi = (0.00364 * (wPI ** 3.35) + 4 * wPI + 11) * 32.44 * math.exp(0.0186 * wPI)
Sopt = (6.752 * (wPI ** 0.147) + 78) / 100

# Datos de entrada y calculados-D60

In [151]:
# Datos de entrada del d60 tmi 
D60 = 0.1
b_D60 = 7.5
# Datos calculados del d60 tmi
a_D60 = 0.8627 * (D60 ** -0.751)
c_D60 = 0.1772 * math.log(D60) + 0.7734
hr_D60 = a_D60 / (D60 + 0.00097)

# Datos de entrada Alfa, beta, gamma, delta (Para calcular las tablas de suelos finos y gruesos)

In [152]:
alpha = 0.3000
beta = 921.4080  
gamma = 150.9908 
delta = 29.8440

# TABLA TMI- MENSUAL

In [153]:
TMI_tabla = pd.DataFrame(
    {
        "mes": [
            "enero",
            "febrero",
            "marzo",
            "abril",
            "mayo",
            "junio",
            "julio",
            "agosto",
            "septiembre",
            "octubre",
            "noviembre",
            "diciembre",
        ],           
    }
)

In [154]:
# TEMPERATURA ES UN DATO DE ENTRADA, SE DEBE PODER MODIFICAR EN LA
TMI_tabla["T(°C)"] = [26.0, 26.4, 26.2, 26.1, 26.1, 26.0, 26.8, 27.7, 27.5, 26.6, 25.7, 25.6]

In [155]:
TMI_tabla["Pi(mm)"] = [182.9, 160.0, 216.3, 192.8, 164.6, 75.4, 38.9, 32.4, 133.2, 229.7, 286.9, 240.8]

In [156]:
TMI_tabla["i"] = (TMI_tabla["T(°C)"] / 5) ** 1.514

In [188]:
# Este I no  esta aproximado aqui, en excel se aproxima
suma_i = TMI_tabla["i"].sum()
suma_i

np.float64(148.98327199557005)

In [186]:
# Este alfa promedio no me da con los decimales que da en el excel
Alfa_promedio = (0.000000675 * suma_i**3 
                - 0.0000771 * suma_i**2 
                + 0.01792 * suma_i 
                + 0.49239)
Alfa_promedio

np.float64(3.6829710699842244)

In [159]:
TMI_tabla["ETPsc(mm/mes)"] = 16 * (10 * TMI_tabla["T(°C)"] / suma_i) ** Alfa_promedio

In [160]:
TMI_tabla["d"] = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [161]:
TMI_tabla["ETPcrr(mm/mes)"] = TMI_tabla["ETPsc(mm/mes)"] * 12 / 12 * TMI_tabla["d"] / 30

In [162]:
TMI_tabla["TMI (1)"] = 100 * (TMI_tabla["Pi(mm)"] / TMI_tabla["ETPcrr(mm/mes)"] - 1)

In [163]:
TMI_tabla["TMI (2)"] = 75 * (TMI_tabla["Pi(mm)"] / TMI_tabla["ETPcrr(mm/mes)"] - 1) + 10

In [164]:
TMI_tabla

,mes,T(°C),Pi(mm),i,ETPsc(mm/mes),d,ETPcrr(mm/mes),TMI (1),TMI (2)
0,enero,26.0,182.9,12.134701,124.392554,31,128.538972,42.291475,41.718606
1,febrero,26.4,160.0,12.418460,131.587494,28,122.814994,30.277253,32.707939
2,marzo,26.2,216.3,12.276302,127.953186,31,132.218292,63.593098,57.694823
3,abril,26.1,192.8,12.205432,126.163720,30,126.163720,52.817308,49.612981
4,mayo,26.1,164.6,12.205432,126.163720,31,130.369177,26.256838,29.692628
5,junio,26.0,75.4,12.134701,124.392554,30,124.392554,-39.385439,-19.539080
6,julio,26.8,38.9,12.704439,139.080938,31,143.716969,-72.932911,-44.699683
7,agosto,27.7,32.4,13.355919,157.072349,31,162.308094,-80.037964,-50.028473
8,septiembre,27.5,133.2,13.210191,152.935797,30,152.935797,-12.904629,0.321528
9,octubre,26.6,229.7,12.561173,135.296426,31,139.806307,64.298739,58.224054


 # Calculo de tabla TMI- PARA SUELOS FINOS

In [165]:
TMI_tabla_finos = pd.DataFrame(
    {
        "mes": [
            "enero",
            "febrero",
            "marzo",
            "abril",
            "mayo",
            "junio",
            "julio",
            "agosto",
            "septiembre",
            "octubre",
            "noviembre",
            "diciembre",
        ],           
    }
)

In [166]:
TMI_tabla_finos["h(1)"] = alpha * (np.exp(beta / (TMI_tabla["TMI (1)"] + gamma)) + delta)

In [167]:
TMI_tabla_finos["h(2)"] = alpha * (np.exp(beta / (TMI_tabla["TMI (2)"] + gamma)) + delta)

In [168]:
TMI_tabla_finos["Ch(h1)"] = 1 - np.log(1 + TMI_tabla_finos["h(1)"] / hr_wpi) / np.log(1 + 1000000 / hr_wpi)

In [169]:
TMI_tabla_finos["Ch(h2)"] = 1 - np.log(1 + TMI_tabla_finos["h(2)"] / hr_wpi) / np.log(1 + 1000000 / hr_wpi)

In [170]:
TMI_tabla_finos["Sr(1)"] = TMI_tabla_finos["Ch(h1)"] / ((np.log(np.exp(1) + (TMI_tabla_finos["h(1)"] / a_wpi) ** b_wpi) ** c_wpi))

In [171]:
TMI_tabla_finos["Sr(2)"] = TMI_tabla_finos["Ch(h2)"] / ((np.log(np.exp(1) + (TMI_tabla_finos["h(2)"] / a_wpi) ** b_wpi)) ** c_wpi)


In [172]:
TMI_tabla_finos["Mr 1/Mropt(Finos)"] = 10 ** (
    -0.3123 +
    (0.3 - -0.3123) /
    (1 + np.exp(
        np.log(-0.3 / -0.3123) + 6.8157 * (TMI_tabla_finos["Sr(1)"] - Sopt)
    ))
)

In [173]:
TMI_tabla_finos["Mr 2/Mropt(Finos)"] = 10 ** (
    -0.3123 +
    (0.3 - (-0.3123)) /
    (1 + np.exp(
        np.log(-0.3 / -0.3123) + 6.8157 * (TMI_tabla_finos["Sr(2)"] - Sopt)
    ))
)

In [174]:
TMI_tabla_finos

,mes,h(1),h(2),Ch(h1),Ch(h2),Sr(1),Sr(2),Mr 1/Mropt(Finos),Mr 2/Mropt(Finos)
0,enero,44.228734,44.732196,0.998881,0.998869,0.943554,0.942952,0.869699,0.870919
1,febrero,57.336333,54.189134,0.998551,0.998630,0.928249,0.931853,0.901493,0.893858
2,marzo,30.929313,33.765001,0.999217,0.999145,0.959880,0.956333,0.837633,0.844436
3,abril,36.530300,38.595454,0.999076,0.999023,0.952908,0.950372,0.851092,0.856073
4,mayo,63.249120,58.138675,0.998402,0.998531,0.921597,0.927337,0.915817,0.903438
5,junio,1164.090760,341.076283,0.972393,0.991525,0.538583,0.725265,1.774570,1.413449
6,julio,40151.400998,1754.368590,0.631158,0.959682,0.184188,0.483548,1.972787,1.836885
7,agosto,130914.479329,2767.013102,0.411551,0.939537,0.104607,0.428348,1.982121,1.883137
8,septiembre,246.112042,141.306058,0.993850,0.996447,0.773925,0.846709,1.287360,1.095166
9,octubre,30.622189,33.489421,0.999225,0.999152,0.960267,0.956676,0.836898,0.843774


 # Calculo de tabla TMI- PARA SUELOS GRUESOS

In [175]:
TMI_tabla_gruesos = pd.DataFrame(
    {
        "mes": [
            "enero",
            "febrero",
            "marzo",
            "abril",
            "mayo",
            "junio",
            "julio",
            "agosto",
            "septiembre",
            "octubre",
            "noviembre",
            "diciembre",
        ],           
    }
)

In [176]:
TMI_tabla_gruesos["h(1)"]= alpha * (np.exp(beta / (TMI_tabla["TMI (1)"] + gamma)) + delta)

In [177]:
TMI_tabla_gruesos["h(2)"] = alpha * (np.exp(beta / (TMI_tabla["TMI (2)"] + gamma)) + delta)

In [178]:
TMI_tabla_gruesos["Ch(h1)"] = 1 - np.log(1 + TMI_tabla_gruesos["h(1)"] / hr_D60) / np.log(1 + 1000000 / hr_D60)

In [179]:
TMI_tabla_gruesos["Ch(h2)"] = 1 - np.log(1 + TMI_tabla_gruesos["h(2)"] / hr_D60) / np.log(1 + 1000000 / hr_D60)

In [180]:
TMI_tabla_gruesos["Sr(1)"] = TMI_tabla_gruesos["Ch(h1)"] / ((np.log(np.exp(1) + (TMI_tabla_gruesos["Ch(h1)"] / a_D60) ** b_D60)) ** c_D60)

In [181]:
TMI_tabla_gruesos["Sr(2)"] = TMI_tabla_gruesos["Ch(h2)"] / ((np.log(np.exp(1) + (TMI_tabla_gruesos["Ch(h2)"] / a_D60)**b_D60))**c_D60)

In [182]:
TMI_tabla_gruesos["Mr 1/Mropt(Gruesos)"] = 10 ** (
    -0.3123 + 
    (0.3 - -0.3123) / 
    (1 + np.exp(
        np.log(-0.3 / -0.3123) + 
        6.8157 * (TMI_tabla_gruesos["Sr(1)"] - Sopt)
    ))
)

In [183]:
TMI_tabla_gruesos["Mr 2/Mropt(Gruesos)"] = 10 ** (-0.3123 + (0.3 - -0.3123) / (1 + np.exp(np.log(-0.3 / -0.3123) + 6.8157 * (TMI_tabla_finos["Sr(2)"] - Sopt))))

In [184]:
TMI_tabla_gruesos

,mes,h(1),h(2),Ch(h1),Ch(h2),Sr(1),Sr(2),Mr 1/Mropt(Gruesos),Mr 2/Mropt(Gruesos)
0,enero,44.228734,44.732196,0.934464,0.933917,0.934464,0.933917,0.888385,0.870919
1,febrero,57.336333,54.189134,0.921118,0.924165,0.921118,0.924164,0.916862,0.893858
2,marzo,30.929313,33.765001,0.950099,0.946556,0.950099,0.946555,0.856613,0.844436
3,abril,36.530300,38.595454,0.943216,0.940793,0.943216,0.940792,0.870385,0.856073
4,mayo,63.249120,58.138675,0.915632,0.920356,0.915632,0.920355,0.928916,0.903438
5,junio,1164.090760,341.076283,0.675513,0.789792,0.675513,0.789792,1.532038,1.413449
6,julio,40151.400998,1754.368590,0.323300,0.635607,0.323300,0.635607,1.938481,1.836885
7,agosto,130914.479329,2767.013102,0.204494,0.590759,0.204494,0.590759,1.969503,1.883137
8,septiembre,246.112042,141.306058,0.817926,0.862217,0.817926,0.862216,1.170386,1.095166
9,octubre,30.622189,33.489421,0.950491,0.946895,0.950490,0.946894,0.855840,0.843774
